#Aggregate Demand , District 1: Outside the Selected region, District 2: Selected Region

##Exports 2*2 aggregated matrix


In [22]:
from IPython.display import display
import os
import numpy as py
import pandas as pd
my_modeller = inro.modeller.Modeller()
emmebank = my_modeller.emmebank
desktop = my_modeller.desktop

#I/O
main_directory = os.path.dirname(os.path.dirname(my_modeller.desktop.project.path)).replace("\\","/")
import_path = os.path.join(main_directory, "emme_project\Database").replace("\\","/")
output_path = os.path.join(main_directory, "output").replace("\\","/")
file_name_input = os.path.join(import_path, "xref_taz13_jur.csv")
file_name_input2 = os.path.join(import_path, "SL_query.csv")

In [23]:
## Read selected demand matrices for a region listed in query and add them to get the aggregated demand of that region
## Then transform it to a 2*2 matrix for internal-external trips the selected region. Using cross refrenced table to 
## identify internal and external TAZs
CR_data = pd.read_csv(file_name_input,sep=',')
query = pd.read_csv(file_name_input2,sep=',')

suffixes = query['Query']
periods = ["EA", "AM", "MD", "PM", "EV"]

for s in suffixes :
    s = str(s)
    agg=[]
    matrix_id_list=[]
#    for p in periods :
    for matrix in emmebank.matrices():
        if matrix.type == "FULL" and "Selected demand" in matrix.description and s in matrix.description :
            matrix_id_list.append(matrix.id)

    mat=[]
    for i in matrix_id_list :
        mat.append(emmebank.matrix(id=i).get_numpy_data())
    demand = reduce(lambda x,y: x+y,  mat)
    demand = pd.DataFrame(demand)
    group_I = CR_data[CR_data['Suffix']== s ]['zone_id']
    zones = [emmebank.scenario(100).zone_numbers] 
    group_E = set(zones[0]) - set(group_I)
    index_I = group_I-1   
    index_E = py.array((map(int,group_E) )) -1
    II=demand.iloc[(index_I)].iloc[:,index_I].values.sum()
    IE=demand.iloc[index_I].iloc[:,index_E].values.sum()
    EI=demand.iloc[index_E].iloc[:,index_I].values.sum()
    EE=demand.iloc[index_E].iloc[:,index_E].values.sum()
    aggregated_demand = pd.DataFrame ({ 'col1' :[EE,IE] , 'col2' : [EI,II] })

    file_name2 = "SL_Agg_SLDaily_" + s +".csv"
    dir_output2 = os.path.join(output_path, file_name2)
    aggregated_demand.to_csv(dir_output2, header = False, index= False)
    TAZ_list = pd.DataFrame({"TAZ" : group_I}) 
    TAZ_list.to_csv( output_path + "/xref_taz13_jur_" + s + ".csv", header = False, index= False)

    